# 🎙️ Karakalpak VITS TTS Model Training on Google Colab

This notebook provides a complete pipeline for fine-tuning a VITS (Variational Inference Text-to-Speech) model for Karakalpak language using the MMS (Massively Multilingual Speech) architecture.

## 📋 Overview
- **Model**: MMS-TTS (VITS architecture)
- **Language**: Karakalpak (kaa)
- **Dataset**: HuggingFace dataset `nickoo004/karakalpak-tts-speaker1`
- **Training Time**: ~20-30 minutes on Colab GPU
- **✅ Works with Private Repositories** - No authentication needed!

## 🚀 Steps:
1. Environment Setup & Dependencies
2. Download Repository (ZIP method - works with private repos!)
3. Dataset Loading from HuggingFace
4. Model Preparation
5. Training Configuration
6. Model Training
7. Inference & Testing
8. Model Saving & Upload

## 1️⃣ Environment Setup & GPU Check

In [ ]:
# Check GPU availability
import torch
print(f"🔍 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow. Please enable GPU in Runtime > Change runtime type")

## 2️⃣ Install Required Dependencies

In [ ]:
%%bash
# Install core dependencies
pip install -q transformers>=4.35.1 datasets[audio]>=2.14.7 accelerate>=0.24.1
pip install -q matplotlib wandb tensorboard Cython
pip install -q scipy librosa soundfile

echo "✅ Dependencies installed successfully!"

## 3️⃣ Download Repository & Build Monotonic Align

This will download the training code from GitHub (works with private repositories!)

In [ ]:
import os
import zipfile
import urllib.request
import shutil

repo_name = "my-vits-finetuner-karakalpak"

# IMPORTANT: Update this URL if your main branch is named differently
# For 'main' branch: /archive/refs/heads/main.zip
# For 'master' branch: /archive/refs/heads/master.zip
zip_url = "https://github.com/NursultanMRX/my-vits-finetuner-karakalpak/archive/refs/heads/main.zip"

if not os.path.exists(repo_name):
    print(f"📥 Downloading repository as ZIP (no authentication needed)...")
    zip_path = "repo.zip"
    
    try:
        # Download the zip file
        urllib.request.urlretrieve(zip_url, zip_path)
        print("✅ Downloaded successfully!")
        
        # Extract the zip file
        print("📦 Extracting files...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(".")
        
        # The extracted folder has '-main' or '-master' suffix
        # Try both possible names
        extracted_name = None
        for suffix in ['-main', '-master']:
            possible_name = f"{repo_name}{suffix}"
            if os.path.exists(possible_name):
                extracted_name = possible_name
                break
        
        if extracted_name:
            # Rename to remove the branch suffix
            shutil.move(extracted_name, repo_name)
            print(f"✅ Renamed '{extracted_name}' to '{repo_name}'")
        else:
            raise Exception(f"Could not find extracted directory (expected {repo_name}-main or {repo_name}-master)")
        
        # Clean up zip file
        os.remove(zip_path)
        print("✅ Repository ready!")
        
    except Exception as e:
        print(f"❌ Error downloading repository: {e}")
        print("\n⚠️ Troubleshooting:")
        print("1. Check if the repository exists and you have access")
        print("2. Verify the branch name (main or master) in the URL above")
        print("3. Try downloading manually and uploading to Colab")
        raise
else:
    print("✅ Repository already exists!")

# Change to repository directory
if os.path.exists(repo_name) and os.path.isdir(repo_name):
    os.chdir(repo_name)
    print(f"📂 Current directory: {os.getcwd()}")
    
    # List some files to verify
    files = os.listdir('.')
    print(f"\n📋 Repository contains {len(files)} items")
    print(f"   Key files present: {', '.join([f for f in ['run_vits_finetuning.py', 'monotonic_align', 'utils'] if f in files])}")
else:
    raise Exception(f"❌ Directory '{repo_name}' does not exist!")

In [ ]:
# Alternative: Use Git Clone (only works for PUBLIC repos or with authentication)
# Uncomment and use this instead if you prefer git clone

# import subprocess
# repo_url = "https://github.com/NursultanMRX/my-vits-finetuner-karakalpak.git"
# repo_name = "my-vits-finetuner-karakalpak"
# 
# if not os.path.exists(repo_name):
#     result = subprocess.run(['git', 'clone', repo_url], capture_output=True, text=True)
#     if result.returncode != 0:
#         print(f"Git clone failed: {result.stderr}")
#     else:
#         print("✅ Repository cloned successfully!")
# 
# os.chdir(repo_name)

In [ ]:
%%bash
# Build the Cython monotonic alignment module (CRITICAL for fast training)
echo "🔨 Building monotonic alignment module..."

# Check if we're in the right directory
if [ ! -d "monotonic_align" ]; then
    echo "❌ Error: monotonic_align directory not found!"
    echo "Current directory: $(pwd)"
    echo "Please make sure the repository was cloned correctly."
    exit 1
fi

cd monotonic_align

# Create the monotonic_align subdirectory if it doesn't exist
mkdir -p monotonic_align

# Build the Cython extension
python setup.py build_ext --inplace

if [ $? -eq 0 ]; then
    echo "✅ Monotonic align built successfully!"
else
    echo "❌ Failed to build monotonic align!"
    exit 1
fi

cd ..

## 4️⃣ HuggingFace Authentication (Optional but Recommended)

**Note**: This is required if you want to:
- Push your trained model to HuggingFace Hub
- Access private datasets

Get your token from: https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login

# Login to HuggingFace (you'll be prompted for your token)
# If you don't want to push to hub, you can skip this step
try:
    notebook_login()
    print("✅ Successfully logged in to HuggingFace!")
except Exception as e:
    print(f"⚠️ Login skipped or failed: {e}")
    print("You can continue training, but won't be able to push to Hub")

## 5️⃣ Load and Explore Dataset from HuggingFace

In [ ]:
from datasets import load_dataset

# Load the Karakalpak TTS dataset from HuggingFace
dataset_name = "nickoo004/karakalpak-tts-speaker1"

print(f"📊 Loading dataset: {dataset_name}")
dataset = load_dataset(dataset_name)

print("\n✅ Dataset loaded successfully!")
print(f"\n📈 Dataset structure:")
print(dataset)

# Show a sample from the dataset
if 'train' in dataset:
    sample = dataset['train'][0]
    print("\n🔍 Sample from dataset:")
    print(f"  - Text: {sample.get('text', 'N/A')}")
    print(f"  - Audio file: {sample.get('audio_file', 'N/A')}")
    print(f"  - Speaker: {sample.get('speaker_name', 'N/A')}")
    print(f"\n📊 Total training samples: {len(dataset['train'])}")

## 6️⃣ Prepare or Download the Base Model with Discriminator

For Karakalpak (kaa), we need to either:
- Use an existing checkpoint with discriminator (if available)
- Convert the original MMS checkpoint to include discriminator

In [ ]:
import os
from huggingface_hub import hf_hub_download, list_repo_files

# Configuration
model_name_or_path = "facebook/mms-tts-kaa"  # Base MMS model for Karakalpak
local_model_dir = "./mms-tts-kaa-with-discriminator"

# Check if we already have a model with discriminator on HuggingFace
# You can replace this with your own model if you've already created one
try:
    # Try to check if a pre-converted model exists
    test_model = "nickoo004/mms-tts-kaa-with-discriminator"
    files = list_repo_files(test_model)
    if 'discriminator.pth' in files or 'pytorch_model.bin' in files:
        print(f"✅ Found existing model with discriminator: {test_model}")
        model_name_or_path = test_model
    else:
        print(f"⚠️ Model exists but discriminator not found. Will convert.")
        raise Exception("Need to convert")
except:
    print(f"\n📝 No pre-converted model found. Converting base MMS model...")
    print(f"   Base model: {model_name_or_path}")
    
    # Convert the model to include discriminator
    # This only needs to be done ONCE per language
    !python convert_original_discriminator_checkpoint.py \
        --language_code kaa \
        --pytorch_dump_folder_path {local_model_dir}
    
    model_name_or_path = local_model_dir
    print(f"\n✅ Model converted and saved to: {model_name_or_path}")

print(f"\n🎯 Model ready for training: {model_name_or_path}")

## 7️⃣ Configure Training Parameters

These settings are optimized for fine-tuning on a single speaker dataset.

In [ ]:
import json

# Training configuration based on finetune_karakalpak.json
# Adjust these parameters as needed for your use case

training_config = {
    # Model and Dataset
    "model_name_or_path": model_name_or_path,
    "dataset_name": "nickoo004/karakalpak-tts-speaker1",
    
    # Output directories
    "output_dir": "./mms-tts-kaa-finetuned-speaker1",
    "overwrite_output_dir": True,
    
    # HuggingFace Hub (set to False if you don't want to push)
    "push_to_hub": False,  # Set to True if you want to upload to Hub
    "hub_model_id": "your-username/mms-tts-kaa-finetuned-speaker1",  # Change this!
    
    # Dataset columns
    "audio_column_name": "audio_file",
    "text_column_name": "text",
    "speaker_id_column_name": "speaker_name",
    "filter_on_speaker_id": "Speaker_1",  # Filter to use only Speaker_1
    "override_speaker_embeddings": True,
    
    # Audio filtering
    "max_duration_in_seconds": 20.0,
    "min_duration_in_seconds": 1.0,
    
    # Training hyperparameters
    "num_train_epochs": 150,  # Reduce for faster testing (e.g., 50)
    "per_device_train_batch_size": 4,
    "learning_rate": 2e-5,
    "warmup_ratio": 0.01,
    "gradient_accumulation_steps": 1,
    "gradient_checkpointing": False,
    "group_by_length": False,
    
    # Training flags
    "do_train": True,
    "do_eval": False,
    
    # Loss weights (GAN training)
    "weight_disc": 3.0,
    "weight_fmaps": 1.0,
    "weight_gen": 1.0,
    "weight_kl": 1.5,
    "weight_duration": 1.0,
    "weight_mel": 35.0,
    
    # Optimization
    "fp16": True,  # Use mixed precision for faster training
    "seed": 42,
    
    # Logging and saving
    "logging_steps": 10,
    "save_steps": 500,
    "save_total_limit": 2,
}

# Save config to file
config_path = "./training_config_colab.json"
with open(config_path, 'w') as f:
    json.dump(training_config, f, indent=4)

print("✅ Training configuration created!")
print("\n📋 Key settings:")
print(f"  - Model: {training_config['model_name_or_path']}")
print(f"  - Dataset: {training_config['dataset_name']}")
print(f"  - Epochs: {training_config['num_train_epochs']}")
print(f"  - Batch size: {training_config['per_device_train_batch_size']}")
print(f"  - Learning rate: {training_config['learning_rate']}")
print(f"  - Output: {training_config['output_dir']}")
print(f"  - Push to Hub: {training_config['push_to_hub']}")
print(f"\n📄 Config saved to: {config_path}")

## 8️⃣ Start Training! 🚀

This will fine-tune the VITS model on your Karakalpak dataset.

**Note**: Training will take approximately 20-30 minutes on a GPU (T4 or better).

If you encounter memory issues, reduce `per_device_train_batch_size` to 2.

In [ ]:
# Run training using accelerate
!accelerate launch run_vits_finetuning.py {config_path}

## 9️⃣ Test the Trained Model - Inference

Let's generate some speech with your fine-tuned model!

In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch
import scipy.io.wavfile
from IPython.display import Audio, display
import numpy as np

# Load the fine-tuned model
model_path = training_config['output_dir']

print(f"📥 Loading model from: {model_path}")
model = VitsModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"✅ Model loaded successfully on {device}!")

In [ ]:
# Test with Karakalpak text samples
test_texts = [
    "Sálem, qalaysız?",  # Hello, how are you?
    "Múgálim jáqsı.",    # The teacher is good.
    "Men oqıp atırman.",  # I am studying.
]

print("🎤 Generating speech samples...\n")

for i, text in enumerate(test_texts, 1):
    print(f"\n{'='*60}")
    print(f"Sample {i}: {text}")
    print('='*60)
    
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    # Generate speech
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get waveform
    waveform = outputs.waveform[0].cpu().numpy()
    
    # Save audio file
    output_file = f"output_sample_{i}.wav"
    scipy.io.wavfile.write(
        output_file,
        rate=model.config.sampling_rate,
        data=waveform
    )
    
    print(f"💾 Saved to: {output_file}")
    
    # Play audio in notebook
    display(Audio(waveform, rate=model.config.sampling_rate))

print("\n✅ All samples generated successfully!")

## 🔟 Custom Text Generation

Try your own Karakalpak text!

In [ ]:
# Enter your custom Karakalpak text here
custom_text = "Sálem, bul men!"  # Change this to your text

print(f"📝 Generating speech for: {custom_text}\n")

inputs = tokenizer(custom_text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

waveform = outputs.waveform[0].cpu().numpy()

# Save
output_file = "custom_output.wav"
scipy.io.wavfile.write(
    output_file,
    rate=model.config.sampling_rate,
    data=waveform
)

print(f"✅ Generated and saved to: {output_file}\n")

# Play
display(Audio(waveform, rate=model.config.sampling_rate))

## 1️⃣1️⃣ Push Model to HuggingFace Hub (Optional)

Share your model with the community!

In [ ]:
# Only run this if you want to upload your model to HuggingFace Hub
# Make sure you've logged in earlier and set a proper hub_model_id

push_to_hub = False  # Set to True to enable pushing

if push_to_hub:
    hub_model_id = training_config['hub_model_id']
    
    if hub_model_id and "your-username" not in hub_model_id:
        print(f"📤 Pushing model to HuggingFace Hub: {hub_model_id}")
        
        model.push_to_hub(hub_model_id)
        tokenizer.push_to_hub(hub_model_id)
        
        print(f"\n✅ Model successfully pushed to: https://huggingface.co/{hub_model_id}")
    else:
        print("⚠️ Please set a valid hub_model_id (replace 'your-username' with your HF username)")
else:
    print("ℹ️ Skipping push to Hub (set push_to_hub=True to enable)")

## 1️⃣2️⃣ Download Trained Model Files

Download your trained model to use locally or deploy elsewhere.

In [ ]:
# Create a zip file of the trained model
import shutil

output_dir = training_config['output_dir']
zip_filename = "mms-tts-kaa-finetuned"

print(f"📦 Creating zip archive of trained model...")
shutil.make_archive(zip_filename, 'zip', output_dir)

print(f"\n✅ Model archived to: {zip_filename}.zip")
print(f"📥 You can download it from the Files panel on the left")
print(f"   Size: {os.path.getsize(f'{zip_filename}.zip') / (1024*1024):.2f} MB")

# List files in the output directory
print(f"\n📂 Model files:")
for file in os.listdir(output_dir):
    file_path = os.path.join(output_dir, file)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path) / (1024*1024)
        print(f"  - {file}: {size:.2f} MB")

## 📊 Training Summary & Next Steps

### What we accomplished:
✅ Set up the training environment
✅ Loaded the Karakalpak dataset from HuggingFace
✅ Prepared/converted the MMS-TTS model with discriminator
✅ Fine-tuned the model on Karakalpak speech data
✅ Generated test audio samples
✅ Saved the trained model

### Next Steps:
1. **Experiment with hyperparameters**: Try different learning rates, batch sizes, or epochs
2. **Use more data**: Add more speakers or audio samples to improve quality
3. **Deploy**: Use the model in a web app or mobile application
4. **Share**: Upload to HuggingFace Hub to share with the community

### Useful Resources:
- [VITS Paper](https://arxiv.org/abs/2106.06103)
- [MMS Paper](https://arxiv.org/abs/2305.13516)
- [HuggingFace VITS Docs](https://huggingface.co/docs/transformers/model_doc/vits)
- [Transformers TTS Guide](https://huggingface.co/docs/transformers/tasks/text-to-speech)

### 🎉 Congratulations on training your Karakalpak TTS model!

## 🔧 Troubleshooting

### Common Issues:

**1. Repository Download Fails:**
- Check if you have internet connection
- Verify the repository exists: https://github.com/NursultanMRX/my-vits-finetuner-karakalpak
- If branch name is different, update the `zip_url` (change 'main' to 'master' if needed)
- As a last resort, manually download the ZIP and upload to Colab Files panel

**2. Out of Memory (OOM) Error:**
- Reduce `per_device_train_batch_size` to 2 or 1
- Enable `gradient_checkpointing: true`
- Ensure you're using a GPU runtime (Runtime > Change runtime type)

**3. Training is slow:**
- Make sure GPU is enabled (should see T4/V100/A100 in GPU check)
- Ensure `fp16: true` is set
- Check that monotonic_align was built correctly

**4. Model quality is poor:**
- Increase `num_train_epochs` (try 200-300)
- Check dataset quality and text alignment
- Ensure audio samples are clean and well-recorded
- Try adjusting loss weights (especially `weight_mel`)

**5. Dataset loading fails:**
- Verify dataset exists: https://huggingface.co/datasets/nickoo004/karakalpak-tts-speaker1
- Check internet connection
- Try restarting the runtime

**6. Can't push to Hub:**
- Make sure you ran `notebook_login()` successfully
- Verify your HuggingFace token has write permissions
- Check that `hub_model_id` doesn't have 'your-username' in it

**7. Monotonic Align Build Fails:**
- Make sure Cython is installed: `!pip install Cython`
- Check that numpy is installed: `!pip install numpy`
- Try restarting runtime and running from the beginning